<a href="https://colab.research.google.com/github/TALeonard/19ma573thomasleonard/blob/master/src/hw7_vasicek_calibration_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1: Verify the solution.

Verify the explicit form of $r_{t}$ given by

$$r_{t} = r_{0}e^{-\kappa t} + \mu(1 - e^{-\kappa t}) + \sigma e^{-\kappa t}\int_{0}^{t}e^{-\kappa s} dW_{s}$$

is the solution to the original Vasicek model, that being

***UNKNOWN EQUATION***

**Work goes here**

Part 2: Design pricing engine of ZCB P(0,T) using above explicit formula. First, import modules.

In [0]:
import numpy as np
import scipy.stats as ss

In [0]:
def zcb_price_explicit(T,kappa,mu,sigma,r0):
  #Compute B(0,T).
  B = (1 - np.exp(-1 * kappa * T))/kappa
  
  #Now, compute A(0,T).
  A = ((mu - ((sigma**2)/(2*(kappa**2)))) * (B - T)) - (((sigma**2)/(4*kappa))*(B**2))
  
  #Given these, compute A - B*r0.
  diff = A - (B * r0)
  
  #Price is the exp(diff).
  return np.exp(diff)

Part 3: Design alternative engine using exact sampling. The integral inside the expectation follows a normal distribution with

$$\mu_{r} = \mu T + (r_{0} - \mu)\frac{1-e^{-\kappa T}}{\kappa}$$

$$\sigma^{2}_{r} = \frac{\sigma^{2}}{2\kappa^{3}}(2\kappa T - 3 + 4e^{-\kappa T} - e^{-2 \kappa T})$$

In [0]:
def zcb_price_exact_sample(T,kappa,mu,sigma,r0,num_path):
  #First, compute the mean and variance of the exponent.
  mu_r = (mu*T) + ((r0 - mu)*(1-np.exp(-1*kappa*T))/kappa)
  var_r = ((sigma**2)/(2*(kappa**3)))*((2*kappa*T) - 3 + (4*np.exp(-1*kappa*T)) - np.exp(-2*kappa*T))
  
  #With this, we can compute (num_path) many iterations of the integral.
  #Generate random values for R, the random variable at hand.
  r = np.random.normal(mu_r,var_r,num_path)
  
  #Create "realized" values for e^(-R(T))
  r_output = np.exp(-1 * r)
  
  #Return the mean of the realized values.
  return r_output.mean()

Part 4: Compute ZCB P(0,1) and Libor L(0,1) using the parameters outlined in the assignment, through both engines. No number of paths is given, so I will use 1000.

In [0]:
'''=======paras======='''
theta = [.1, .05, .003, .03]
kappa, mu, sigma, r0 = theta
num_path = 1000

In [17]:
formula_price = zcb_price_explicit(1,kappa,mu,sigma,r0)
sample_price = zcb_price_exact_sample(1,kappa,mu,sigma,r0,num_path)

print("ZCB P(0,1) price by formula is " + str(formula_price))
print("ZCB P(0,1) price by exact sampling is " + str(sample_price))

ZCB P(0,1) price by formula is 0.9695084475425054
ZCB P(0,1) price by exact sampling is 0.9695070862513501


In [18]:
# Use above values to compute LIBOR L(0,1) values.
# L(t,T) = (100/(T-t)) * ((1/P(t,T)) - 1)

def libor_price(T,P):
  #WARNING: THIS ASSUMES STARTING TIME IS 0, I.E. L(0,T).
  #P is price of relevant ZCB.
  factor_1 = 100/(T)
  factor_2 = (1/P) - 1
  return factor_1 * factor_2

print("LIBOR L(0,1) price using ZCB formula price is " + str(libor_price(1,formula_price)))
print("LIBOR L(0,1) price using ZCB sample price is " + str(libor_price(1,sample_price)))

LIBOR L(0,1) price using ZCB formula price is 3.145052787810565
LIBOR L(0,1) price using ZCB sample price is 3.145197614444717


Part 5: 10-term swap rates with term length 1/2 year, i.e. $s(t = 0, T = 5, N = 10)$

This is just a simple computation. Define swap rate pricing engine, and produce results via both engines.

In [0]:
def swap_price(T,N,kappa,mu,sigma,r0,num_path):
  #WARNING: THIS ASSUMES STARTING TIME IS 0, I.E. L(0,T).
  #Define delta.
  delta = T/N
  
  #Create array to store P(t,t+j*delta) terms. Compute using both engines.
  price_explicit = np.zeros(N)
  price_sample = np.zeros(N)
  
  #Iteratively fill price matrix.
  for j in range(N):
    price_explicit[j] = zcb_price_explicit((j+1)*delta,kappa,mu,sigma,r0)
    price_sample[j] = zcb_price_exact_sample((j+1)*delta,kappa,mu,sigma,r0,num_path)
  
  #Produce numerators and denominators.
  num_explicit = 1 - price_explicit[N-1]
  denom_explicit = price_explicit.sum()
  
  num_sample = 1 - price_sample[N-1]
  denom_sample = price_sample.sum()
  
  s_explicit = 100 * num_explicit / denom_explicit
  
  s_sample = 100 * num_sample / denom_sample
  
  print("10-term swap rate with term length 1/2 year using the formula is " + str(s_explicit))
  print("10-term swap rate with term length 1/2 year using exact sampling is " + str(s_sample))

In [22]:
#Now, compute.

swap_price(5,10,kappa,mu,sigma,r0,num_path)

10-term swap rate with term length 1/2 year using the formula is 1.7209106981949385
10-term swap rate with term length 1/2 year using exact sampling is 1.7222767596811837
